## 2유형 [4] Stellar-Classification

In [25]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import numpy as np
import pandas as pd
X_test = pd.read_csv("./datasets/Part2/stellar_X_test.csv")
X_train = pd.read_csv("./datasets/Part2/stellar_X_train.csv")
y_train = pd.read_csv("./datasets/Part2/stellar_y_train.csv")

# 사용자 코딩

# 1. 데이터 전처리
# 1) 데이터 확인 -> 결측치 없고, 모두 수치형
# print(X_train.info())

# 2) 기술통계량
# print(X_train.describe())

# 3) 이상치 제거 ★
y_train = y_train.loc[X_train.u != -9999]
X_train = X_train.loc[X_train.u != -9999]

X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

# print(X_train.describe())

# 4) redshift 데이터의 분위를 살펴보고, 파생변수 만들어주기
# print(X_train['redshift'].quantile([q/100 for q in range(90, 101)]))
# 전체의 2%만 3을 넘음

# 파생변수 추가
X_train['redshift_upper3'] = np.where(X_train['redshift']>3, 1, 0)
X_test['redshift_upper3'] = np.where(X_test['redshift']>3, 1, 0)

# 컬럼 추가
COL_DEL = []
COL_NUM = ['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'redshift']
COL_CAT = ['run_ID', 'rerun_ID', 'cam_col', 'field_ID', 'plate', 'redshift_upper3']
COL_Y = ['galaxy']

X_train[COL_CAT] = X_train[COL_CAT].astype(str)
X_test[COL_CAT] = X_test[COL_CAT].astype(str)

X_train = X_train.drop(COL_DEL, axis=1)
X_test = X_test.drop(COL_DEL, axis=1)

# 5) 수치형 데이터가 유효한지 살펴보기
train_df = pd.concat([X_train, y_train], axis=1)

# for col in COL_NUM:
#     print('-'*80)
#     print(col)
#     print(train_df.groupby(COL_Y)[col].describe(), end='\n\n')
# 수치형 중 의미있어 보이는 변수 : redshift

# 6) 범주형 데이터가 유효한지 살펴보기
# for col in COL_CAT:
#     print(train_df.groupby(col, as_index=False)[COL_Y].mean().sort_values(by=COL_Y, ascending=False, end='\n\n')
# 범주형 중 의미있어 보이는 변수 : cam_col, redshift_upper3

# 7) 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
X = pd.concat([X_train, X_test])

for col in COL_CAT:
    le = LabelEncoder()
    le.fit(X[col])
    X_train[col] = le.transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

# --------------------

# 2. 모델 구축
# 1) 데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train)

# 2) 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_tr[COL_NUM] = scaler.fit_transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.fit_transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.fit_transform(X_test[COL_NUM])

# 3) 분류모델 적용 (레이블 인코딩을 했으므로 랜덤포레스트와 XGBoost)
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(X_tr, y_tr.values.ravel())

from xgboost import XGBClassifier

model_xgb = XGBClassifier()
model_xgb.fit(X_tr, y_tr.values.ravel())

# --------------------

# 3. 모델 평가 (ROC_AUC_Score)
# 1) 예측값 계산
y_pred_rf = model_rf.predict_proba(X_val)
y_pred_xgb = model_xgb.predict_proba(X_val)

# 2) 평가 점수
from sklearn.metrics import roc_auc_score

score_rf = roc_auc_score(y_val, y_pred_rf[:, 1])
score_xgb = roc_auc_score(y_val, y_pred_xgb[:, 1])

# print(score_rf, score_xgb)
# 높을수록 좋은 모델 : xgboost

# 3) 변수 중요도 확인
# print(pd.DataFrame({'feature': X_tr.columns, 'fi_rf': model_rf.feature_importances_, 
#                     'fi_xgb': model_xgb.feature_importances_}))

# --------------------

# 4. 제출 파일로 저장
pred = model_xgb.predict_proba(X_test)[:, 1]
pd.DataFrame({'galaxy':pred}).to_csv('./res/04.csv', index=False)

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'gender': pred}).to_csv('003000000.csv', index=False)